In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from symbol import Symbol
from index import Index
import seaborn as sns
from sklearn.preprocessing import scale

sns.set_style("dark")

%matplotlib inline

In [7]:
symbol = Symbol(symbol_list='federalbnk', start=2016)
index = Index(index_list='nifty_50')

In [4]:
sdata = symbol.get_symbol_data('federalbnk', start=2016)
idata = index.get_index_hist(start=2015)
len(sdata), len(idata)

D:\Dev\Projects\Learn\iPynb\HashtagMe\Stock Analysis\source\symbol.py:262: UserWarning: Invalid type of data requested. Returning returns data
  'Invalid type of data requested. Returning returns data'


NameError: name 'index' is not defined

In [43]:
data = sdata.copy()
data = sdata.join(idata)
data['date'] = [x for x in range(len(data))]
data = data.join(data.log_returns.shift(-1).rename('fut_returns'))
data['close'] = data.ix[0, 'close'] * np.exp(data.log_returns)
data = data.drop(['open', 'high', 'low', 'last', 'prev_close', 'vwap', 'turnover',
                  'daily_volatility', 'simple_returns', 'log_returns', 'pct_deliverble'], axis=1)
corr = data.corr()
len(data), len(data.columns)

(488, 5)

In [44]:
indices = np.where(abs(corr) > 0.8)
indices = [(corr.index[x], corr.columns[y]) for x, y in zip(*indices) if x != y and x < y]
indices

[]

In [45]:
for index in indices:
    try:
        corr = corr.drop(index[1], axis=1)
        corr = corr.drop(index[1])
    except:
        pass
corr

,close,volume,nifty_50,date,fut_returns
close,1.000000,-0.045926,0.005856,-0.039686,0.011050
volume,-0.045926,1.000000,-0.071327,0.145137,0.030155
nifty_50,0.005856,-0.071327,1.000000,-0.125860,-0.075781
date,-0.039686,0.145137,-0.125860,1.000000,-0.038596
fut_returns,0.011050,0.030155,-0.075781,-0.038596,1.000000


In [46]:
data = data.dropna()
print(len(data))
X = data.ix[:, 0:-1]
y = data.ix[:, -1].fillna(0)
y = pd.cut(y*100, bins=[-25, 0, 25], labels=[-1, 1]).astype(int)

485


In [47]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [49]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [40]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [57]:
clf.score(scale(X_train), (y_train)), clf.score(scale(X_test), (y_test))

(0.50964187327823696, 0.5)

In [60]:
clf.predict(X_train).sum()

-7

In [61]:
y_train.sum()

-7

In [ ]:
y.min()